In [38]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import lightgbm as lgb

from sklearn.metrics import mean_squared_error, confusion_matrix, roc_auc_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing, metrics


In [3]:
# pickle ファイルから読み込み
train_pkl = pd.read_pickle('./train.pk4')

In [4]:
train_pkl.shape

(72326, 10)

In [5]:
# object 型は削除
train_pkl.drop(['grade', 'purpose'], axis=1, inplace=True)

In [6]:
train_pkl.head()

,id,loan_amnt,term,interest_rate,employment_length,credit_score,application_type,loan_status
66580,66580,1124.604497,3,11.560074,0,658.607427,0,1
9030,9030,2215.993885,3,18.280527,10,715.903149,0,1
175788,175788,1028.815459,3,13.962523,10,698.254596,0,0
59630,59630,2398.256532,5,18.302722,2,657.878810,0,0
88393,88393,1566.226772,3,14.668532,0,659.409219,0,1


In [7]:
# train_pk2 = train_pkl.copy()
train_pkl.shape

(72326, 8)

## 訓練データとテストデータに分割

In [8]:
# 訓練とテストデータに分割
train, test = train_test_split(train_pkl, test_size=0.2, random_state=42)

In [9]:
# ターゲットと特徴量の分割
train_X = train.iloc[:, 1:-1]
train_y = train.loan_status

In [10]:
# ターゲットと特徴量の分割
test_X = test.iloc[:, 1:-1]
test_y = test.loan_status

In [11]:
# LightGBMのモデルへデータセットの初期化セット
train_data = lgb.Dataset(train_X, label=train_y)
test_data =  lgb.Dataset(test_X, label=test_y, reference=train_data)

## LightGBM

In [ ]:
# パラメータ
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'verbose': 2
}

In [ ]:
# 訓練
gbm = lgb.train(
    params,
    train_data,
    valid_sets=test_data,
    num_boost_round=1000,
    verbose_eval=50,
    early_stopping_rounds=100
)


In [12]:
# scikit-learn経由で
gbm2 = lgb.LGBMClassifier(objective='binary', n_estimators=1000, seed=42)

In [13]:
gbm2.fit(train_X, train_y, eval_set=[(test_X, test_y)], verbose=50)

[50]	valid_0's binary_logloss: 0.602874
[100]	valid_0's binary_logloss: 0.602993
[150]	valid_0's binary_logloss: 0.604212
[200]	valid_0's binary_logloss: 0.605526
[250]	valid_0's binary_logloss: 0.606873
[300]	valid_0's binary_logloss: 0.608007
[350]	valid_0's binary_logloss: 0.609419
[400]	valid_0's binary_logloss: 0.610621
[450]	valid_0's binary_logloss: 0.612324
[500]	valid_0's binary_logloss: 0.613875
[550]	valid_0's binary_logloss: 0.614889
[600]	valid_0's binary_logloss: 0.61631
[650]	valid_0's binary_logloss: 0.617647
[700]	valid_0's binary_logloss: 0.6191
[750]	valid_0's binary_logloss: 0.620786
[800]	valid_0's binary_logloss: 0.622035
[850]	valid_0's binary_logloss: 0.623198
[900]	valid_0's binary_logloss: 0.624685
[950]	valid_0's binary_logloss: 0.62589
[1000]	valid_0's binary_logloss: 0.626914


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1000, n_jobs=-1, num_leaves=31, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, seed=42,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)

In [14]:
# preds = gbm2.predict(test_X, num_iteration=gbm2.best_iteration)
preds = gbm2.predict(test_X)


In [15]:
preds = gbm2.predict_proba(test_X)

In [16]:
y_pred = np.where(preds < 0.5, 0, 1)

In [17]:
pred_y = np.argmax(y_pred, axis=1)

In [18]:
confusion_matrix(test_y, pred_y)

array([[2597, 3344],
       [1705, 6820]], dtype=int64)

In [19]:
test_y.sum(), pred_y.sum()

(8525, 10164)

In [20]:
accuracy_score(test_y, pred_y)

0.6509746992948984

In [21]:
precision_score(test_y, pred_y)

0.670995670995671

In [22]:
recall_score(test_y, pred_y)

0.8

In [23]:
f1_score(test_y, pred_y)

0.7298410829899941

In [39]:
roc_auc_score(test_y, pred_y)

ValueError: Found input variables with inconsistent numbers of samples: [14466, 26900]

In [ ]:
fpr, tpr, thr = roc_curve(test_y, pred_y)

In [ ]:
auc = metrics.auc(fpr, tpr)

In [ ]:
auc

In [ ]:
plt.plot(fpr, tpr, label="ROC Curve %.2f"%auc)
plt.legend()
plt.xlabel('FPR')
plt.ylabel("TPR")
plt.grid()
plt.show()

## 検証データで実行

In [25]:
# 検証データ読み込み
valid = pd.read_pickle('./test.pk4')

In [26]:
# ID の保存
valid_pass = valid.id.values

In [27]:
# valid_X = valid.iloc[:, 1:]
valid_X = valid.copy()

In [28]:
valid_X.drop(['id', 'grade', 'purpose'], axis=1, inplace=True)

In [29]:
valid_X.shape, train_X.shape

((26900, 6), (57860, 6))

In [30]:
pred_valid_y = gbm2.predict(valid_X)

In [31]:
pred_valid_y.shape

(26900,)

In [32]:
preds = gbm2.predict_proba(valid_X)

In [33]:
y_pred = np.where(preds < 0.5, 0, 1)

In [34]:
pred_y = np.argmax(y_pred, axis=1)

In [35]:
# type(valid_pass), type(pred_valid_y)

In [36]:
result_df = pd.DataFrame(pred_y, valid_pass, columns=['y'])

In [37]:
result_df.to_csv("./LGBM_4_1.csv", header=False)

In [ ]:
result_df